In [ ]:
import logging, os

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')

import warnings

warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly
import re
from scipy.io import wavfile
from scipy.fftpack import fft
import cv2
import os
import librosa
import librosa.display
import IPython.display as ipd
from glob import glob
import skimage
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tqdm import tqdm
import tempfile
import plotly.graph_objects as go

In [ ]:
from gudhi import RipsComplex
import gudhi as gd
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import pairwise_distances
from scipy.stats import wasserstein_distance
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from gudhi.representations import Landscape
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from gudhi.representations import PersistenceImage 
from pathos.multiprocessing import ProcessingPool as Pool
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

from pydub import AudioSegment
from pydub.utils import which
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
AudioSegment.converter = "/opt/homebrew/bin/ffmpeg"
AudioSegment.ffprobe = "/opt/homebrew/bin/ffprobe"

In [ ]:
myPath = 'audio-emotions/1/Emotions/'
def loadFiles(aDirectory):
    myEmotionsSounds = []
    for myFile in tqdm(os.listdir(aDirectory)):
        if myFile.endswith(".wav"):
            sound = AudioSegment.from_file(os.path.join(aDirectory, myFile))
            myEmotionsSounds.append(sound)
    return myEmotionsSounds 

In [ ]:
def getWaveform(aPath, aPathName):
    mySignal, mySampleRate = librosa.load(aPath)
    plt.figure(figsize=(14, 5))
    librosa.display.waveshow(mySignal, sr=mySampleRate)
    plt.title(F'The {aPathName} Sound Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()
    print(F"The {aPathName} Sound: ")
    sound = ipd.Audio(mySignal, rate=mySampleRate)
    return sound

In [ ]:
def doMelSpectrogram(aSoundFiles, aFixLength = False):
    mySampleRate = 44100  
    myMelSpectrograms = []
    for i in tqdm(range(len(aSoundFiles))):
        myAudioSegment = aSoundFiles[i]
        myTempFile = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        myTempFilePath = myTempFile.name
        myTempFile.close()
        myAudioSegment.export(myTempFilePath, format="wav")
        mySignal, mySampleRate = librosa.load(myTempFilePath, sr=mySampleRate)
        os.remove(myTempFilePath)
        mel_spectrogram = librosa.feature.melspectrogram(y=mySignal, sr=mySampleRate, n_mels=128) 
        mel_spectrogram_dbs = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)
        if aFixLength:
            mel_spectrogram_dbs = librosa.util.fix_length(mel_spectrogram_dbs, size=128, axis=1)
        myMelSpectrograms.append(mel_spectrogram_dbs)
    return myMelSpectrograms

In [ ]:
def computeMfccs(aDbs, aNumMfcc=13):
    myMfccList = []
    for i in tqdm(range(len(aDbs))):
        myMfcc = librosa.feature.mfcc(S=aDbs[i], n_mfcc=aNumMfcc)
        myMfccList.append(myMfcc)
    return myMfccList

In [ ]:
def showMelSpectrogram(aMelSpectrogram):  
    myDfMelSpectrogram = pd.DataFrame(aMelSpectrogram)
    plt.figure(figsize=(10, 4))
    sns.heatmap(myDfMelSpectrogram, cmap='plasma', xticklabels=10, yticklabels=10)
    plt.title(f'Mel Spectrogram from an angry sound  (dB)')
    plt.xlabel('Time (s)')
    plt.ylabel('Mel Frequency Bin')
    plt.show()

In [ ]:
def showMfcc(aMfcc):
    librosa.display.specshow(aMfcc, x_axis='time')
    plt.colorbar()
    plt.title('MFCC From an Angry Sound')
    plt.show()

In [ ]:
def show3DPlot(aMelSpectrogram):
    fig = go.Figure(data=[
        go.Surface(z=aMelSpectrogram.T, colorscale='viridis')
    ])
    fig.update_layout(
        title='Mel Spectrogram from and angry sound 3-D plot (dB)',
        scene=dict(
            xaxis=dict(title='Time (s)'),
            yaxis=dict(title='Mel Frequency Bin'),
            zaxis=dict(title='Amplitude (dB)')
        )
    )
    fig.show()

## Preprocessing: Getting MFCCs and Spectrograms

In [ ]:
myAngrySounds = loadFiles(myPath + '/Angry/')
print("Angry sounds loaded successfully")
myDisgustedSounds = loadFiles(myPath + '/Disgusted/')
print("Disgusted sounds loaded successfully")
myFearfulSounds = loadFiles(myPath + '/Fearful/')
print("Fearful sounds loaded successfully")
myHappySounds = loadFiles(myPath + '/Happy/')
print("Happy sounds loaded successfully")
myNeutralSounds = loadFiles(myPath + '/Neutral/')
print("Neutral sounds loaded successfully")
mySadSounds = loadFiles(myPath + '/Sad/')
print("Sad sounds loaded successfully")
mySurprisedSounds = loadFiles(myPath + 'Suprised/')
print("Suprised sounds loaded successfully")

In [ ]:
myMelSpectrogramsAngry = doMelSpectrogram(myAngrySounds)
myMelSpectrogramsDisgusted = doMelSpectrogram(myDisgustedSounds)
myMelSpectrogramsFearful = doMelSpectrogram(myFearfulSounds)
myMelSpectrogramsHappy = doMelSpectrogram(myHappySounds)
myMelSpectrogramsNeutral = doMelSpectrogram(myNeutralSounds)
myMelSpectrogramsSurprised = doMelSpectrogram(mySurprisedSounds)
myMelSpectrogramsSad = doMelSpectrogram(mySadSounds)


In [ ]:
myMfccAngry = computeMfccs(myMelSpectrogramsAngry)
myMfccDisgusted = computeMfccs(myMelSpectrogramsDisgusted)
myMfccFearful = computeMfccs(myMelSpectrogramsFearful)
myMfccHappy = computeMfccs(myMelSpectrogramsHappy)
myMfccNeutral = computeMfccs(myMelSpectrogramsNeutral)
myMfccSurprised = computeMfccs(myMelSpectrogramsSurprised)
myMfccSad = computeMfccs(myMelSpectrogramsSad)

In [ ]:
getWaveform(myPath+'/Angry/03-01-05-01-01-01-01.wav','Angry')


In [ ]:
showMelSpectrogram(myMelSpectrogramsAngry[0])


In [ ]:
showMfcc(myMfccAngry[0])


In [ ]:
show3DPlot(myMelSpectrogramsAngry[0])

## Computing Homologies!

In [ ]:
def euclideanMetric(x, y):
    return np.linalg.norm(x - y) 
def timeBasedEuclideanMetric(x, y):
    return np.linalg.norm(x[0] - y[0]) / (2) + np.linalg.norm(x[1:] - y[1:]) / (2)


In [ ]:
def computerPersistenceDiagrams(aMatrix, aMaxLength, aMaxDim, aMetric, aPlot, aNormalize):
    # Preprocess matrix
    if aNormalize:
        myTimeRow = np.arange(aMatrix.shape[1]) 
        myNormalizedMatrix = (aMatrix - np.min(aMatrix)) / (np.max(aMatrix) - np.min(aMatrix))
        myTimeRow = myTimeRow / aMatrix.shape[1]
        myNewArr = np.vstack([myTimeRow, myNormalizedMatrix])
    else:
        myNewArr = aMatrix
    
    # Compute distance matrix
    myDistanceMatrix = squareform(pdist(myNewArr.T, metric=aMetric))
    myMaxLength = min(np.abs(myDistanceMatrix).max().max() / 3, aMaxLength)

    # Compute persistences
    myRipsComplex = RipsComplex(points=myNewArr.T, max_edge_length=myMaxLength, distance_matrix=myDistanceMatrix)
    mySimplexTree = myRipsComplex.create_simplex_tree(max_dimension=aMaxDim)
    mySimplexTree.compute_persistence()
    myPersistence = mySimplexTree.persistence()

    # Compute bd matrix
    myBd = []
    for i in range(aMaxDim):
        myBdMatrix = np.array([
            [birth, death] for dim, (birth, death) in myPersistence if dim == i and death != float('inf')
        ])
        myBd.append(myBdMatrix)

    # Plot if needed
    if aPlot:
        gd.plot_persistence_diagram(mySimplexTree.persistence())

    return myBd

In [ ]:
def computePersistanceFromDb(aDb, aMaxLength, aMaxDim, aMetric, aPlot, aMaxDiagrams, aNormalize):
    myMat = []
    for i, myCurr in enumerate(aDb):
        if i == aMaxDiagrams:
            break
        print(f"Computing persitance diagram for {i}")
        myPersistence = computerPersistenceDiagrams(aMatrix=myCurr, aMaxLength=aMaxLength, aMaxDim=aMaxDim, aMetric=aMetric, aPlot=aPlot, aNormalize=aNormalize)
        myMat.append(myPersistence)
    return myMat

In [ ]:
def getPersitanceHeatMap(aArr, aMax, aMaxDim, aPlot = False, aShape = 32):
    myHeatmapArr = []
    for myBdMatrix in aArr:
        print(myBdMatrix)
        myPersistenceImage = PersistenceImage(
            bandwidth= aMax / 50,
            weight=lambda x: np.sqrt((x[1] - x[0])**2), 
            resolution=[aShape, aShape],    
            im_range=[0, aMax, 0, aMax]   
        )

        for i in range(aMaxDim):
            # If any persistence bd matrix found then compute heatmap
            if i < len(myBdMatrix) and len(myBdMatrix[i]) > 0:
                myHeatMap = myPersistenceImage.fit_transform([myBdMatrix[i]])[0]
                myHeatMap2d = myHeatMap.reshape(aShape, aShape)
                myHeatmapArr.append(myHeatMap2d)
                if aPlot:
                    print(myHeatMap2d.shape)
                    plt.figure()
                    plt.imshow(myHeatMap2d, cmap='hot', origin='lower')
                    plt.colorbar(label='Scaled Density')
                    plt.title('Persistence Image')
                    plt.xlabel('Birth')
                    plt.ylabel('Persistence')
                    plt.show()
            else:
                myHeatmapArr.append(np.zeros((aShape, aShape)))
    return myHeatmapArr

In [ ]:
myPersistenceT = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsAngry[0], aMaxLength=0.8, aMaxDim=2, aMetric=timeBasedEuclideanMetric, aPlot=True, aNormalize=True)
myPersistenceW = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsAngry[0], aMaxLength=2, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)
myPersistenceE = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsAngry[0], aMaxLength=80, aMaxDim=2, aMetric=euclideanMetric, aPlot=True, aNormalize=False)
myPersistenceWM = computerPersistenceDiagrams(aMatrix=myMfccAngry[0], aMaxLength=10, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)

heatmaps = getPersitanceHeatMap(aArr=[myPersistenceT], aMax=0.8, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceW], aMax=2, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceE], aMax=80, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceWM], aMax=10, aMaxDim=2, aPlot=True)

In [ ]:
myPersistenceT = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsDisgusted[0], aMaxLength=0.8, aMaxDim=2, aMetric=timeBasedEuclideanMetric, aPlot=True, aNormalize=True)
myPersistenceW = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsDisgusted[0], aMaxLength=2, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)
myPersistenceE = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsDisgusted[0], aMaxLength=80, aMaxDim=2, aMetric=euclideanMetric, aPlot=True, aNormalize=False)
myPersistenceWM = computerPersistenceDiagrams(aMatrix=myMfccDisgusted[0], aMaxLength=10, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)

heatmaps = getPersitanceHeatMap(aArr=[myPersistenceT], aMax=0.8, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceW], aMax=2, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceE], aMax=80, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceWM], aMax=10, aMaxDim=2, aPlot=True)

In [ ]:
def parallelComputePersitence(args):
    return computePersistanceFromDb(*args)

In [ ]:
def saveResults(aPattern, aResults, aMaxDim):
    for i in range(len(aResults)):
        for k in range(len(aResults[i])):
            for j in range(aMaxDim):
                np.save(f'./savefiles/{aPattern}_{i}_{k}_{j}.npy', aResults[i][k][j])

In [ ]:
def computeHeatMaps(aResults, aMaxDim, aMaxDist, aName):
    myNewHeat = []
    for i in range(len(aResults)):
        print(f'Computing {i}')
        myHeat = getPersitanceHeatMap(aResults[i], aMax=aMaxDist, aMaxDim=aMaxDim)
        myNewHeat.append(myHeat)
        for j in range(len(myNewHeat[i])):
            print(j)
            np.save(f'./savefiles/{aName}Heat_{i}_{j}.npy', myNewHeat[i][j])

In [ ]:
maxdim = 2
metrics = [timeBasedEuclideanMetric, wasserstein_distance, euclideanMetric]
scaled = [True, False, False]
maxdists = [0.8, 2, 80]
names = ['timeMetric', 'wasserstein', 'euclidean']
for metric, scale, maxdist, name in zip(metrics, scaled, maxdists, names):
    args = [
        (myMelSpectrogramsAngry, maxdist, maxdim, metric, False, len(myMelSpectrogramsAngry), scale),
        (myMelSpectrogramsDisgusted, maxdist, maxdim, metric, False, len(myMelSpectrogramsDisgusted), scale),
        (myMelSpectrogramsFearful, maxdist, maxdim, metric, False, len(myMelSpectrogramsFearful), scale),
        (myMelSpectrogramsHappy, maxdist, maxdim, metric, False, len(myMelSpectrogramsHappy), scale),
        (myMelSpectrogramsNeutral, maxdist, maxdim, metric, False, len(myMelSpectrogramsNeutral), scale),
        (myMelSpectrogramsSurprised, maxdist, maxdim, metric, False, len(myMelSpectrogramsSurprised), scale),
        (myMelSpectrogramsSad, maxdist, maxdim, metric, False, len(myMelSpectrogramsSad), scale)
    ]
    with Pool() as pool:
        results = pool.map(parallelComputePersitence, args)
    saveResults(aPattern=name, aResults=results, aMaxDim=maxdim)
    computeHeatMaps(aResults=results, aMaxDim=maxdim, aMaxDist=maxdist, aName=name)
    print('To the next!')

In [ ]:
maxDim = 2
maxdist = 10
args = [
    (myMfccAngry, maxdist, maxDim, wasserstein_distance, False, len(myMfccAngry), False),
    (myMfccDisgusted, maxdist, maxDim, wasserstein_distance, False, len(myMfccDisgusted), False),
    (myMfccFearful, maxdist, maxDim, wasserstein_distance, False, len(myMfccFearful), False),
    (myMfccHappy, maxdist, maxDim, wasserstein_distance, False, len(myMfccHappy), False),
    (myMfccNeutral, maxdist, maxDim, wasserstein_distance, False, len(myMfccNeutral), False),
    (myMfccSurprised, maxdist, maxDim, wasserstein_distance, False, len(myMfccSurprised), False),
    (myMfccSad, maxdist, maxDim, wasserstein_distance, False, len(myMfccSad), False)
]
with Pool() as pool:
    results = pool.map(parallelComputePersitence, args)
saveResults(aPattern='wassersteinMfcc', aResults=results, aMaxDim=maxDim)
computeHeatMaps(aResults=results, aMaxDim=maxDim, aMaxDist=maxdist, aName='wassersteinMfcc')


In [ ]:
myMelSpectrogramsAngry = doMelSpectrogram(myAngrySounds, True)
myMelSpectrogramsDisgusted = doMelSpectrogram(myDisgustedSounds, True)
myMelSpectrogramsFearful = doMelSpectrogram(myFearfulSounds, True)
myMelSpectrogramsHappy = doMelSpectrogram(myHappySounds, True)
myMelSpectrogramsNeutral = doMelSpectrogram(myNeutralSounds, True)
myMelSpectrogramsSurprised = doMelSpectrogram(mySurprisedSounds, True)
myMelSpectrogramsSad = doMelSpectrogram(mySadSounds, True)

In [ ]:
for i in range(len(myMelSpectrogramsAngry)):
    np.save(f'./savefiles/mel_spectrogram_dbs_angry_{i}.npy', myMelSpectrogramsAngry[i])
for i in range(len(myMelSpectrogramsDisgusted)):
    np.save(f'./savefiles/mel_spectrogram_dbs_disgusted_{i}.npy', myMelSpectrogramsDisgusted[i])
for i in range(len(myMelSpectrogramsFearful)):
    np.save(f'./savefiles/mel_spectrogram_dbs_fearful_{i}.npy', myMelSpectrogramsFearful[i])
for i in range(len(myMelSpectrogramsHappy)):
    np.save(f'./savefiles/mel_spectrogram_dbs_happy_{i}.npy', myMelSpectrogramsHappy[i])
for i in range(len(myMelSpectrogramsSurprised)):
    np.save(f'./savefiles/mel_spectrogram_dbs_surprised_{i}.npy', myMelSpectrogramsSurprised[i])
for i in range(len(myMelSpectrogramsSad)):
    np.save(f'./savefiles/mel_spectrogram_dbs_sad_{i}.npy', myMelSpectrogramsSad[i])
for i in range(len(myMelSpectrogramsNeutral)):
    np.save(f'./savefiles/mel_spectrogram_dbs_neutral_{i}.npy', myMelSpectrogramsNeutral[i])

In [ ]:
folder = './savefiles'

def findFilesFromPattern(pattern):
    pattern = re.compile(pattern + r'_(\d+)_(\d+)_(\d+)\.npy')
    heatmaps_dict = {}
    for filename in os.listdir(folder):
        match = pattern.match(filename)
        if match:
            i, j, k = map(int, match.groups())
            filepath = os.path.join(folder, filename)
            data = np.load(filepath)
            
            if i not in heatmaps_dict:
                heatmaps_dict[i] = []
            
            while len(heatmaps_dict[i]) <= j:
                heatmaps_dict[i].append([])
            
            while len(heatmaps_dict[i][j]) <= k:
                heatmaps_dict[i][j].append(None)
            
            heatmaps_dict[i][j][k] = data
    return [heatmaps_dict[i] for i in sorted(heatmaps_dict.keys())]

In [ ]:
import os

folder = "./savefiles"

for filename in os.listdir(folder):
    # print(filename)
    if "_Actor_" in filename:
        new_name = filename.replace("_Actor_", "_")
        os.rename(os.path.join(folder, filename), os.path.join(folder, new_name))